In [1]:
from functools import partial
from multiprocessing.dummy import Pool
from subprocess import call
import os.path as osp
from general_tools.in_out.basics import files_in_subdirs, create_dir

from tf_lab.data_sets.shape_net import snc_category_to_synth_id

from geo_tool.in_out.soup import load_wavefront_obj, write_wavefront_obj

In [2]:
# Parameters
binary = '/orions3-zfs/software/blender-2.74/blender '
binary_arg1 = '/orions4-zfs/projects/optas/Testing/Hao/blank.blend '
binary_arg2 = '--background --python /orions4-zfs/projects/optas/Testing/Hao/render.py '

binary = binary + binary_arg1 + binary_arg2

top_out_image_dir = '/orions4-zfs/projects/optas/DATA/Images/Shape_Net_Core/2015_Summer_OUT/Rendered_Images/'
create_dir(top_out_image_dir)
thread_num = 20

In [3]:
class_name = raw_input('Give me the class type.\n').lower()
syn_id = snc_category_to_synth_id()[class_name]
print syn_id

top_in_dir = '/orions4-zfs/projects/optas/DATA/Meshes/Shape_Net_Core/2015_Summer/' 
top_in_dir = osp.join(top_in_dir, syn_id)

top_out_dir = osp.join(top_out_image_dir, syn_id)

Give me the class type.
car
02958343


In [4]:
search_pattern = '.obj$'
all_in_files = [f for f in files_in_subdirs(top_in_dir, search_pattern)]
print len(all_in_files)

7497


In [ ]:
commands = []
for in_f in all_in_files:
    model_name = in_f.split('/')[-2]
    out_dir = create_dir(osp.join(top_out_dir, model_name))
    
    vertices, faces, normals = load_wavefront_obj(in_f) # this is sequential now
    out_obj = osp.join(out_dir, 'model.obj')
    write_wavefront_obj(out_obj, vertices, faces, normals)
    
    command = '%s %s %s' % (binary, out_obj, out_dir) # Use the new obj to avoid plotting the material.
    commands.append(command)
    
print ('\nTotal number of commands is %d' % (len(commands)))

In [ ]:
pool = Pool(thread_num)
for idx, return_code in enumerate(pool.imap(partial(call, shell=True), commands)):
    if return_code != 0:
        print('command %d of %s failed' % (idx, commands[idx]))

In [28]:
# Delete new obj.
commands = []
for in_f in all_in_files:
    model_name = in_f.split('/')[-2]
    out_dir = create_dir(osp.join(top_out_dir, model_name))
    out_obj = osp.join(out_dir, 'model.obj')        
    command = 'rm -rf %s' % (out_obj)
    commands.append(command)
    
print commands[0]
print ('\nTotal number of commands is %d' % (len(commands)))

rm -rf /scr/optas/DATA/Meshes/Shape_Net_Core/2015_Summer_OUT/Images/04379243/28f702b5c6ccffe7fcf9154182ccb5a4/model.obj

Total number of commands is 8509
